In [4]:
# create a new database from 2 old ones

# flat_new_WECGrid.db has a grid "flat run" that I need to add to temp_new_WECGrid.db

# the grid sim id's are the same so we'll need to update the flat run from 1 -> 2 

flat_db = "./flat_new_WECGrid.db"
temp_db = "./temp_new_WECGrid.db"


In [7]:
        # Copy grid_simulations metadata (update the ID and handle UNIQUE constraint)
        grid_sim_row = source_df[source_df['grid_sim_id'] == old_grid_sim_id].copy()
        grid_sim_row['grid_sim_id'] = new_grid_sim_id
        
        # Check if this combination would violate UNIQUE constraint
        target_df = pd.read_sql("SELECT * FROM grid_simulations", target_conn)
        
        # Check for potential UNIQUE constraint violation
        potential_conflict = target_df[
            (target_df['case_name'] == grid_sim_row['case_name'].iloc[0]) &
            (target_df['psse'] == grid_sim_row['psse'].iloc[0]) &
            (target_df['pypsa'] == grid_sim_row['pypsa'].iloc[0]) &
            (target_df['sim_start_time'] == grid_sim_row['sim_start_time'].iloc[0])
        ]
        
        if len(potential_conflict) > 0:
            print(f"⚠️  UNIQUE constraint conflict detected!")
            print(f"   Existing sim: {potential_conflict['sim_name'].iloc[0]}")
            print(f"   New sim: {grid_sim_row['sim_name'].iloc[0]}")
            
            # Modify sim_start_time slightly to avoid conflict (add 1 second)
            from datetime import datetime, timedelta
            original_time = grid_sim_row['sim_start_time'].iloc[0]
            try:
                dt = datetime.fromisoformat(original_time.replace('T', ' '))
                new_time = dt + timedelta(seconds=1)
                grid_sim_row['sim_start_time'] = new_time.strftime('%Y-%m-%dT%H:%M:%S')
                print(f"   Modified sim_start_time: {original_time} → {grid_sim_row['sim_start_time'].iloc[0]}")
            except:
                # Fallback: just append a suffix to make it unique
                grid_sim_row['sim_start_time'] = f"{original_time}_copy"
                print(f"   Modified sim_start_time: {original_time} → {grid_sim_row['sim_start_time'].iloc[0]}")
        
        grid_sim_row.to_sql('grid_simulations', target_conn, if_exists='append', index=False)
        print(f"Copied grid simulation metadata with new ID: {new_grid_sim_id}")

Checking database integrity...

=== Checking ./flat_new_WECGrid.db ===
✅ Database integrity: OK
✅ Grid simulations table: 1 records
✅ Found 13 tables: grid_simulations, sqlite_sequence, wec_simulations, wec_integrations, psse_bus_results...

=== Checking ./temp_new_WECGrid.db ===
✅ Database integrity: OK
✅ Grid simulations table: 1 records
✅ Found 13 tables: grid_simulations, sqlite_sequence, wec_simulations, wec_integrations, psse_bus_results...

=== Checking ./temp_new_WECGrid.db ===
✅ Database integrity: OK
✅ Grid simulations table: 1 records
✅ Found 13 tables: grid_simulations, sqlite_sequence, wec_simulations, wec_integrations, psse_bus_results...

✅ Both databases are healthy - proceeding with combination...
Created new database: ./combined_WECGrid.db
Source DB: ./flat_new_WECGrid.db
Target DB: ./combined_WECGrid.db
New grid_sim_id for source data: 2
Found 1 grid simulation(s) in source database
Error during database combination: UNIQUE constraint failed: grid_simulations.case_na

IntegrityError: UNIQUE constraint failed: grid_simulations.case_name, grid_simulations.psse, grid_simulations.pypsa, grid_simulations.sim_start_time

In [13]:
# Alternative approach: Create a fresh database with current schema (no UNIQUE constraint)
# and copy ALL data from both source databases including WEC-Sim runs

def create_fresh_combined_database(flat_db_path, temp_db_path, output_db_path):
    """
    Create a new database with modern schema (no UNIQUE constraints) and copy data from both sources.
    
    This approach creates a completely fresh database using the current WEC-Grid schema,
    which doesn't have the problematic UNIQUE constraint on grid_simulations.
    """
    
    print(f"\n=== Creating Fresh Combined Database ===")
    
    # Remove output database if it exists
    if os.path.exists(output_db_path):
        os.remove(output_db_path)
        print(f"Removed existing {output_db_path}")
    
    # Create fresh database with modern schema (no UNIQUE constraints)
    print("Creating new database with modern schema...")
    output_conn = sqlite3.connect(output_db_path)
    
    # Create the grid_simulations table with NO UNIQUE constraint
    output_conn.execute("""
        CREATE TABLE grid_simulations (
            grid_sim_id INTEGER PRIMARY KEY AUTOINCREMENT,
            sim_name TEXT,
            case_name TEXT NOT NULL,
            psse BOOLEAN DEFAULT FALSE,
            pypsa BOOLEAN DEFAULT FALSE,
            sbase_mva REAL NOT NULL,
            sim_start_time TEXT NOT NULL,
            sim_end_time TEXT,
            delta_time INTEGER,
            notes TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create other required tables (copying structure from one of the source databases)
    source_conn = sqlite3.connect(flat_db_path)
    
    # Get all table creation statements except grid_simulations
    cursor = source_conn.execute("""
        SELECT name, sql FROM sqlite_master 
        WHERE type='table' AND name != 'grid_simulations' AND name != 'sqlite_sequence'
    """)
    
    for table_name, create_sql in cursor.fetchall():
        if create_sql:  # Some system tables might have None SQL
            output_conn.execute(create_sql)
            print(f"Created table: {table_name}")
    
    # Create indexes (except the problematic unique index)
    cursor = source_conn.execute("""
        SELECT name, sql FROM sqlite_master 
        WHERE type='index' AND tbl_name != 'grid_simulations' AND sql IS NOT NULL
    """)
    
    for index_name, index_sql in cursor.fetchall():
        try:
            output_conn.execute(index_sql)
        except sqlite3.Error as e:
            print(f"Note: Could not create index {index_name}: {e}")
    
    # Create safe indexes for grid_simulations (without UNIQUE constraint)
    output_conn.execute("CREATE INDEX idx_grid_sim_time ON grid_simulations(sim_start_time)")
    output_conn.execute("CREATE INDEX idx_grid_sim_case ON grid_simulations(case_name)")
    
    output_conn.commit()
    source_conn.close()
    
    print("✅ Fresh database schema created successfully!")
    
    # First, copy WEC-Sim simulation data (only from temp_db which has WECs)
    print(f"\n=== Copying WEC-Sim Data ===")
    
    wec_sim_id_mapping = {}  # Track old -> new wec_sim_id mappings
    current_wec_sim_id = 0
    
    # Only check temp_db for WEC-Sim data (flat_db has no WECs)
    source_conn = sqlite3.connect(temp_db_path)
    
    # Check if wec_simulations table exists and has data
    try:
        wec_sims_df = pd.read_sql("SELECT * FROM wec_simulations", source_conn)
        
        if not wec_sims_df.empty:
            print(f"Found {len(wec_sims_df)} WEC simulation(s) in temp database")
            
            for _, wec_sim_row in wec_sims_df.iterrows():
                current_wec_sim_id += 1
                old_wec_sim_id = wec_sim_row['wec_sim_id']
                wec_sim_id_mapping[f"{temp_db_path}_{old_wec_sim_id}"] = current_wec_sim_id
                
                print(f"  Copying WEC sim {old_wec_sim_id} → {current_wec_sim_id}: {wec_sim_row['model_type']}")
                
                # Insert wec simulation with new ID
                output_conn.execute("""
                    INSERT INTO wec_simulations 
                    (wec_sim_id, model_type, sim_duration_sec, delta_time,
                     wave_height_m, wave_period_sec, wave_spectrum, wave_class, 
                     wave_seed, simulation_hash, created_at)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (current_wec_sim_id, wec_sim_row['model_type'], wec_sim_row['sim_duration_sec'],
                      wec_sim_row['delta_time'], wec_sim_row.get('wave_height_m'), 
                      wec_sim_row.get('wave_period_sec'), wec_sim_row.get('wave_spectrum'),
                      wec_sim_row.get('wave_class'), wec_sim_row.get('wave_seed'),
                      wec_sim_row.get('simulation_hash'), wec_sim_row['created_at']))
                
                # Copy WEC power results if they exist
                try:
                    wec_power_df = pd.read_sql(
                        "SELECT * FROM wec_power_results WHERE wec_sim_id = ?", 
                        source_conn, params=(old_wec_sim_id,)
                    )
                    
                    if not wec_power_df.empty:
                        # Update wec_sim_id to new value
                        wec_power_df['wec_sim_id'] = current_wec_sim_id
                        
                        # Insert into output database
                        wec_power_df.to_sql('wec_power_results', output_conn, if_exists='append', index=False)
                        print(f"    Copied {len(wec_power_df)} WEC power result rows")
                        
                except Exception as e:
                    print(f"    Note: No WEC power results found: {e}")
        else:
            print(f"No WEC simulations found in temp database")
            
    except Exception as e:
        print(f"No WEC simulation data in temp database: {e}")
    
    source_conn.close()
    
    # Now copy grid simulation data
    print(f"\n=== Copying Grid Simulation Data ===")
    
    databases_to_copy = [
        (temp_db_path, "temp database"),
        (flat_db_path, "flat database")
    ]
    
    current_grid_sim_id = 0
    
    for db_path, db_description in databases_to_copy:
        print(f"\n--- Copying data from {db_description} ---")
        
        source_conn = sqlite3.connect(db_path)
        
        # Get grid simulation metadata
        grid_sims_df = pd.read_sql("SELECT * FROM grid_simulations", source_conn)
        
        for _, sim_row in grid_sims_df.iterrows():
            current_grid_sim_id += 1
            old_sim_id = sim_row['grid_sim_id']
            
            print(f"  Copying grid_sim_id {old_sim_id} → {current_grid_sim_id}: {sim_row['sim_name']}")
            
            # Insert grid simulation with new ID
            output_conn.execute("""
                INSERT INTO grid_simulations 
                (grid_sim_id, sim_name, case_name, psse, pypsa, sbase_mva, 
                 sim_start_time, sim_end_time, delta_time, notes, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (current_grid_sim_id, sim_row['sim_name'], sim_row['case_name'], 
                  sim_row['psse'], sim_row['pypsa'], sim_row['sbase_mva'],
                  sim_row['sim_start_time'], sim_row['sim_end_time'], 
                  sim_row['delta_time'], sim_row['notes'], sim_row['created_at']))
            
            # Copy all related data tables
            result_tables = [
                'psse_bus_results', 'psse_generator_results', 'psse_load_results', 'psse_line_results',
                'pypsa_bus_results', 'pypsa_generator_results', 'pypsa_load_results', 'pypsa_line_results'
            ]
            
            for table in result_tables:
                try:
                    # Check if table exists
                    cursor = source_conn.execute(
                        "SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table,)
                    )
                    if not cursor.fetchone():
                        continue
                    
                    # Get data for this grid_sim_id
                    table_df = pd.read_sql(
                        f"SELECT * FROM {table} WHERE grid_sim_id = ?", 
                        source_conn, params=(old_sim_id,)
                    )
                    
                    if len(table_df) > 0:
                        # Update grid_sim_id to new value
                        table_df['grid_sim_id'] = current_grid_sim_id
                        
                        # Insert into output database
                        table_df.to_sql(table, output_conn, if_exists='append', index=False)
                        print(f"    Copied {len(table_df)} rows from {table}")
                        
                except Exception as e:
                    print(f"    Error copying {table}: {e}")
            
            # Handle wec_integrations with updated IDs (only for temp_db which has WECs)
            try:
                wec_integration_df = pd.read_sql(
                    "SELECT * FROM wec_integrations WHERE grid_sim_id = ?", 
                    source_conn, params=(old_sim_id,)
                )
                
                if not wec_integration_df.empty:
                    for _, integration_row in wec_integration_df.iterrows():
                        old_wec_sim_id = integration_row['wec_sim_id']
                        mapping_key = f"{db_path}_{old_wec_sim_id}"
                        
                        if mapping_key in wec_sim_id_mapping:
                            new_wec_sim_id = wec_sim_id_mapping[mapping_key]
                            
                            output_conn.execute("""
                                INSERT INTO wec_integrations 
                                (grid_sim_id, wec_sim_id, farm_name, bus_location, num_devices, created_at)
                                VALUES (?, ?, ?, ?, ?, ?)
                            """, (current_grid_sim_id, new_wec_sim_id, integration_row['farm_name'],
                                  integration_row['bus_location'], integration_row['num_devices'],
                                  integration_row['created_at']))
                            
                            print(f"    Linked WEC integration: grid_sim {current_grid_sim_id} ↔ wec_sim {new_wec_sim_id}")
                        else:
                            # This is expected for flat_db which has no WEC simulations
                            if db_description == "temp database":
                                print(f"    Warning: Could not find WEC sim mapping for {old_wec_sim_id}")
                            
            except Exception as e:
                # This is expected for flat_db which has no WEC integrations
                if db_description == "temp database":
                    print(f"    Note: No WEC integrations found: {e}")
        
        source_conn.close()
    
    output_conn.commit()
    output_conn.close()
    
    print(f"\n✅ Combined database created successfully: {output_db_path}")
    print(f"Total grid simulations: {current_grid_sim_id}")
    print(f"Total WEC simulations: {current_wec_sim_id}")
    
    return output_db_path

# Create the fresh combined database with WEC-Sim data
output_database = "./fresh_combined_WECGrid.db"
result_db = create_fresh_combined_database(flat_db, temp_db, output_database)


=== Creating Fresh Combined Database ===
Removed existing ./fresh_combined_WECGrid.db
Creating new database with modern schema...
Created table: wec_simulations
Created table: wec_integrations
Created table: psse_bus_results
Created table: psse_generator_results
Created table: psse_load_results
Created table: psse_line_results
Created table: pypsa_bus_results
Created table: pypsa_generator_results
Created table: pypsa_load_results
Created table: pypsa_line_results
Created table: wec_power_results
✅ Fresh database schema created successfully!

=== Copying WEC-Sim Data ===
Found 2 WEC simulation(s) in temp database
  Copying WEC sim 1 → 1: RM3
    Copied 864001 WEC power result rows
  Copying WEC sim 2 → 2: LUPA
    Copied 864001 WEC power result rows
  Copying WEC sim 2 → 2: LUPA
    Copied 864001 WEC power result rows

=== Copying Grid Simulation Data ===

--- Copying data from temp database ---
  Copying grid_sim_id 1 → 1: PSSE-RTS-GMLC: RM3 Farm
    Copied 21312 rows from psse_bus_r

In [14]:
# Verify the fresh combined database with WEC-Sim data
def verify_combined_database(db_path):
    """Verify the combined database has the expected data."""
    conn = sqlite3.connect(db_path)
    
    try:
        print(f"\n=== Verification of {db_path} ===")
        
        # Check grid simulations
        grid_sims = pd.read_sql("SELECT * FROM grid_simulations ORDER BY grid_sim_id", conn)
        print(f"\nGrid Simulations ({len(grid_sims)} total):")
        print(grid_sims[['grid_sim_id', 'sim_name', 'case_name', 'psse', 'pypsa', 'sbase_mva']].to_string(index=False))
        
        # Check WEC simulations
        try:
            wec_sims = pd.read_sql("SELECT * FROM wec_simulations ORDER BY wec_sim_id", conn)
            if not wec_sims.empty:
                print(f"\nWEC Simulations ({len(wec_sims)} total):")
                print(wec_sims[['wec_sim_id', 'model_type', 'sim_duration_sec', 'wave_height_m', 'wave_period_sec']].to_string(index=False))
            else:
                print("\nNo WEC simulations found")
        except Exception as e:
            print(f"\nNo WEC simulation data: {e}")
        
        # Check database schema - verify no UNIQUE constraint
        cursor = conn.execute("""
            SELECT sql FROM sqlite_master 
            WHERE type='table' AND name='grid_simulations'
        """)
        create_statement = cursor.fetchone()
        if create_statement:
            print(f"\n--- Schema Verification ---")
            has_unique = "UNIQUE" in create_statement[0]
            print(f"UNIQUE constraint present: {'❌ YES' if has_unique else '✅ NO'}")
            if not has_unique:
                print("✅ Fresh schema successfully removes problematic constraint!")
        
        # Check data counts for each grid_sim_id
        for grid_sim_id in grid_sims['grid_sim_id']:
            print(f"\n--- Data for grid_sim_id {grid_sim_id} ({grid_sims[grid_sims['grid_sim_id']==grid_sim_id]['sim_name'].iloc[0]}) ---")
            
            # Check result tables
            result_tables = [
                'psse_bus_results', 'psse_generator_results', 'psse_load_results', 'psse_line_results',
                'pypsa_bus_results', 'pypsa_generator_results', 'pypsa_load_results', 'pypsa_line_results'
            ]
            
            for table in result_tables:
                try:
                    cursor = conn.execute(f"SELECT COUNT(*) FROM {table} WHERE grid_sim_id = ?", (grid_sim_id,))
                    count = cursor.fetchone()[0]
                    if count > 0:
                        print(f"  {table}: {count} rows")
                except:
                    print(f"  {table}: table not found or error")
            
            # Check WEC integrations
            try:
                wec_integration_df = pd.read_sql("SELECT * FROM wec_integrations WHERE grid_sim_id = ?", conn, params=(grid_sim_id,))
                if not wec_integration_df.empty:
                    for _, integration in wec_integration_df.iterrows():
                        print(f"  wec_integrations: {integration['farm_name']} at bus {integration['bus_location']} ({integration['num_devices']} devices, wec_sim_id {integration['wec_sim_id']})")
            except:
                pass
        
        # Check WEC power results
        try:
            wec_power_count = pd.read_sql("SELECT wec_sim_id, COUNT(*) as count FROM wec_power_results GROUP BY wec_sim_id", conn)
            if not wec_power_count.empty:
                print(f"\n--- WEC Power Results ---")
                for _, row in wec_power_count.iterrows():
                    print(f"  wec_sim_id {row['wec_sim_id']}: {row['count']} power data points")
        except Exception as e:
            print(f"\nNo WEC power results: {e}")
                
    finally:
        conn.close()

# Verify the fresh combined database
verify_combined_database(result_db)


=== Verification of ./fresh_combined_WECGrid.db ===

Grid Simulations (2 total):
 grid_sim_id                sim_name       case_name  psse  pypsa  sbase_mva
           1 PSSE-RTS-GMLC: RM3 Farm RTS GMLC Hooman     1      0      100.0
           2           PSSE-RTS-GMLC RTS GMLC Hooman     1      0      100.0

WEC Simulations (2 total):
 wec_sim_id model_type  sim_duration_sec  wave_height_m  wave_period_sec
          1        RM3           86400.0            2.5              8.0
          2       LUPA           86400.0            2.5              8.0

--- Schema Verification ---
UNIQUE constraint present: ✅ NO
✅ Fresh schema successfully removes problematic constraint!

--- Data for grid_sim_id 1 (PSSE-RTS-GMLC: RM3 Farm) ---
  psse_bus_results: 21312 rows
  psse_generator_results: 28512 rows
  psse_load_results: 14688 rows
  psse_line_results: 30528 rows
  wec_integrations: WEC-Farm at bus 326 (10 devices, wec_sim_id 1)

--- Data for grid_sim_id 2 (PSSE-RTS-GMLC) ---
  psse_bus_res

In [11]:
# Test loading the combined database with WEC-Grid
import wecgrid

# Create engine and set database path
engine = wecgrid.Engine()
engine.database.set_database_path(result_db)

# Show available simulations
print("=== Available Simulations in Combined Database ===")
grid_sims = engine.database.grid_sims()
print(grid_sims.to_string(index=False))

# Test pulling both simulations
print("\n=== Testing Data Retrieval ===")
for sim_id in grid_sims['grid_sim_id']:
    print(f"\n--- Testing grid_sim_id {sim_id} ---")
    try:
        # Try to pull the simulation
        if grid_sims[grid_sims['grid_sim_id'] == sim_id]['psse'].iloc[0]:
            psse_data = engine.database.pull_sim(sim_id, software='psse')
            print(f"  PSS®E data loaded: {len(psse_data.bus)} buses, {len(psse_data.gen)} generators")
            
        if grid_sims[grid_sims['grid_sim_id'] == sim_id]['pypsa'].iloc[0]:
            pypsa_data = engine.database.pull_sim(sim_id, software='pypsa')
            print(f"  PyPSA data loaded: {len(pypsa_data.bus)} buses, {len(pypsa_data.gen)} generators")
            
    except Exception as e:
        print(f"  Error loading sim_id {sim_id}: {e}")

print(f"\n✅ Combined database successfully created: {result_db}")
print("You can now use this database with your WEC-Grid engine!")

=== Available Simulations in Combined Database ===
 grid_sim_id                sim_name       case_name  psse  pypsa  sbase_mva      sim_start_time        sim_end_time  delta_time                                                                                                                                                                                                                            notes          created_at
           1 PSSE-RTS-GMLC: RM3 Farm RTS GMLC Hooman     1      0      100.0 2025-08-24T00:00:00 2025-08-24T23:55:00         300 RTS-GMLC grid simulation using PSS/E. The simulation was run for 24 hours at 5-minute resolution with no load curve applied. A WEC Farm with 10 RM3 WEC models was included in this simulation using the RM3 WEC-Sim run id = 1. 2025-08-24 20:19:19
           2           PSSE-RTS-GMLC RTS GMLC Hooman     1      0      100.0 2025-08-24T00:00:00 2025-08-24T23:55:00         300                                                     RTS-GMLC grid simula